In [1]:
import string
alphabet = string.ascii_lowercase
bigrams = [a + b for a in alphabet for b in alphabet]

In [2]:
import json
import re
from collections import Counter

# Load the JSON data
with open('cranfield/cran_docs.json', 'r') as file:
    data = json.load(file)

# Create a list to store all the words
all_words = []

# Define a regular expression pattern to match alphanumeric tokens
pattern = re.compile(r'\w+')

# Iterate over each document
for doc in data:
    body = doc['body']
    # Find all alphanumeric tokens in the body text
    words = pattern.findall(body)
    # Append the words to the all_words list
    all_words.extend(words)

# Create a Counter object to count the frequency of each word
word_counts = Counter(all_words)

# Create a set of unique alphanumeric words (Vocabulary)
Vocabulary = set(word_counts.keys())

# Print the Vocabulary set
print("Vocabulary:", Vocabulary)
print("Vocabulary size:", len(Vocabulary))

Vocabulary: {'sedimentation', 'floating', 'advent', '1degree', 'weak', 'amplitudes', 'sharply', 'contracting', 'tribute', 'hoshizaki', '61', 'minutes', 'numerical', 'phases', 'f40umerical', 'acoustics', '38', 'densities', 'streams', 'landahl', 'errors', 'british', 'intermediate', 'outlines', 'sea', 'hydrodynamical', '4000', 'la', 'program', 'screen', 'etc', 'b', 'dy', 'revision', 'nomograph', 'exists', 'foils', 'marked', '104', 'stability', '45degrees', 'we', 'claim', 'oxide', 'from', 'supersonic', 'increasing', 'implement', 'streamwise', 'dashn', '2250', '130', 'circulation', 'puts', '68', '170', 'kinetics', 'preheating', 'concentration', 'hub', 'retains', 'intimately', 'lacking', 'moment', 'working', 'sweepback', 'fluids', '180', 'methanol', 'busemann', 'satellite', 'sectors', 'view', 'fluctuations', '0904', 'probable', 'noninsulated', 'neighborhood', 'poinds', 'crucial', 'balanced', '1300', 'kelley', 'mean', 'constituent', 'collectively', 'preston', '897', 'statements', 'conservativ

In [3]:
import numpy as np
def word_to_vector(word):
    vector = [0] * len(bigrams)
    for i in range(len(word) - 1):
        bigram = word[i:i+2]
        if bigram in bigrams:
            vector[bigrams.index(bigram)] += 1
    return vector

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    if magnitude_vec1 == 0 or magnitude_vec2 == 0:
        return 0  # Prevent division by zero
    return dot_product / (magnitude_vec1 * magnitude_vec2)

In [4]:
# Typo correction for each typo
typos = ['boundery', 'transiant', 'aerplain']
typos_top_five_corrections = {} # Hashmap to store the top five corrections for each typo
for typo in typos:
    typo_vector = word_to_vector(typo)
    similarities = []
    for word in Vocabulary:
        word_vector = word_to_vector(word)
        similarity = cosine_similarity(typo_vector, word_vector)
        similarities.append((word, similarity))
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_five_corrections = similarities[:5]
    typos_top_five_corrections[typo] = top_five_corrections

In [5]:
# Print the top five corrections for each typo along with their cosine similarity truncating to 3 decimal places
for typo, top_five_corrections in typos_top_five_corrections.items():
    print(f"Top five corrections for '{typo}':")
    for correction, similarity in top_five_corrections:
        print(f"  {correction} ({similarity:.3f})")
    print()

Top five corrections for 'boundery':
  bounded (0.772)
  bound (0.756)
  under (0.756)
  unbounded (0.717)
  boundary (0.714)

Top five corrections for 'transiant':
  trans (0.791)
  transient (0.783)
  transit (0.775)
  transcendant (0.702)
  entrant (0.671)

Top five corrections for 'aerplain':
  plain (0.756)
  explain (0.617)
  airplane (0.571)
  explains (0.571)
  explaining (0.570)



In [6]:
def edit_distance(p, q):
    m = len(p)
    n = len(q)
    
    # Create a table to store the edit distances
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Initialize the first row and column of the table
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
    
    # Fill in the rest of the table
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if p[i - 1] == q[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j] + 1,  # Deletion
                               dp[i][j - 1] + 1,  # Insertion
                               dp[i - 1][j - 1] + 1)  # Substitution
    
    # Return the edit distance between p and q
    return dp[m][n]

In [7]:
# calculate the best edit using the edit distance function among the top 5 corrections
best_edits = []
for typo in typos:
    top_five_corrections = typos_top_five_corrections[typo]
    best_edit = min(top_five_corrections, key=lambda x: edit_distance(typo, x[0]))
    best_edits.append((typo, best_edit[0], edit_distance(typo, best_edit[0])))

In [8]:
# Print the best edit for each typo along with the edit distance
for typo, best_edit, distance in best_edits:
    print(f"Best edit for '{typo}': '{best_edit}' (edit distance: {distance})")

Best edit for 'boundery': 'boundary' (edit distance: 1)
Best edit for 'transiant': 'transient' (edit distance: 1)
Best edit for 'aerplain': 'explain' (edit distance: 2)
